In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import torch.nn as nn
import numpy as np
import glob
import learn2learn as l2l
from human_body_prior.tools.omni_tools import copy2cpu as c2c
import warnings
from torch import optim
warnings.filterwarnings("ignore")

import trimesh
from human_body_prior.tools.omni_tools import colors
from human_body_prior.mesh import MeshViewer
from human_body_prior.mesh.sphere import points_to_spheres
from notebook_tools import show_image

In [2]:
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from human_body_prior.body_model.body_model import BodyModel

bm_path = '/home/michalislazarou/PhD/amass/smplh/male/model.npz'
dmpl_path = '/home/michalislazarou/PhD/amass/dmpl/male/model.npz'
npz_bdata_path ='/home/michalislazarou/PhD/amass/datasets/EKUT/234/'
num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_path=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, path_dmpl=dmpl_path).to(comp_device)
faces = c2c(bm.f)

In [4]:
# import meta-dataset
data_list = []
for data_file in glob.glob(os.path.join(npz_bdata_path, '*.npz')):
    bdata = dict(np.load(data_file))
    bdata['poses'] = bdata['poses'][:200,:]
    bdata['trans'] = bdata['trans'][:200,:]
    bdata['dmpls'] = bdata['dmpls'][:200,:]
    data_list.append(bdata)
train_list, test_list = data_list[:70], data_list[65:] 
print(len(data_list), len(train_list), len(test_list))

79 70 14


In [5]:
alpha=0.01
beta=0.001
shots=30
adaptation_steps=1
meta_batch_size = 10
num_iterations = 20000

In [6]:
fId = 100 # frame id of the mocap sequence

root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
dmpls = torch.Tensor(bdata['dmpls'][fId:fId+1]).to(comp_device) # controls soft tissue dynamics

body = bm( pose_body=pose_body, pose_hand=pose_hand, betas=betas, dmpls=dmpls)
body_mesh = trimesh.Trimesh(vertices=c2c(body.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
body_mesh.show()


In [ ]:

maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
loss=nn.MSELoss()
opt = optim.Adam(maml.parameters(), meta_lr)